# Tutorial 9 (in Wk 10): Web Scraping Data

## Introduction
In this tutorial, you will get practical exposure to web scraping techniques.

# EXERCISE 1 - Scraping Data from a Website

In the lecture, we demonstrated how to extract data from a given website using Unix command line tools. In this tutorial, we will learn how to do so with Python.

We will use the following Python libraries:
- **Request**         for interacting with websites and web services
- **Beautiful Soup**  for webpage parsing



Our goal is to load data from the following website that makes available historic data from the British Convict Transportation Register: https://convictrecords.com.au

Let's start by having a look at the actual webpage.
The following code requests the webpage for **the ship 'Batavia'** from this ConvictRecords website (the given URL scheme is specific for this website):

### Let’s try to get a webpage.
### Now, we have a Response object called webpage_source. We can get all the information we need from this object.

In [1]:
import requests
webpage_source = requests.get("https://convictrecords.com.au/ships/batavia/").text
print(webpage_source)

<!DOCTYPE html>
<html>
<head>
    <meta http-equiv="Content-Type" content="text/html; charset=utf-8"/>
    <title>Batavia Convict Ship</title>
        <!-- carabiner is breaking image paths in remote CSS files so loading statically instead -->
    <link href="https://fonts.googleapis.com/css?family=Tenor+Sans" rel="stylesheet" type="text/css"/>
    <link type="text/css" rel="stylesheet" href="https://ajax.googleapis.com/ajax/libs/jqueryui/1/themes/redmond/jquery-ui.css" media="screen"/>
    <link type="text/css" rel="stylesheet" href="https://convictrecords.com.au/assets/cache/1492489223318938f69e24b1400e4395ce08be11bc.css" media="all" />
<link type="text/css" rel="stylesheet" href="https://convictrecords.com.au/assets/cache/1492489223c333a9d41fdb00f9462ec9dc65f32f57.css" media="print" />
        <script type="text/javascript" src="https://convictrecords.com.au/assets/cache/00000000009cfa3c1494016f33081271beb86160d2.js" charset="UTF-8"></script>
    <script type="text/javascript" src="

What you see above as the output of this request is the raw webpage source code.
This is normally parsed and rendered by a web browser as a nice visual webpage.

The language in which this webpage is written, is called **HTML** (the *Hypertext Markup Language*).
In order to extract data from a webpage, we need at least some understanding of HTML, and in particularm that it is describing a tree-like structure of content elements which are rendered by a web browser.

We can interpret the content of this webpage also inside a program by using a so-called **HTML parser**.
For Python, there are several different options. In this tutorial, we are using a parser called *Beautiful Soup*. This parser allows a Python program to extract content of a webpage or a web service by finding specific HTML tags and giving access to the content inside.

Have a look at the following Python code. It shows how the BeautifulSoup library is used to parse the webpage source code and then demonstrates different navigational commands of that library to identify specific tags within a (complex) page:

### Beautiful Soup is a Python library for pulling data out of HTML and XML files. It works with your favorite parser to provide idiomatic ways of navigating, searching, and modifying the parse tree. It commonly saves programmers hours or days of work.

In [2]:
from bs4 import BeautifulSoup

### To parse a document, pass it into the BeautifulSoup constructor
page_content = BeautifulSoup(webpage_source, 'html5lib')

### Beautiful Soup transforms a complex HTML document into a complex tree of Python objects. 
### But you’ll only ever have to deal with about four kinds of objects: Tag, NavigableString, BeautifulSoup, and Comment.

#### A Tag object corresponds to an XML or HTML tag in the original document

#### The simplest way to navigate the parse tree is to say the name of the tag you want. 
### If you want the <title> tag, just say page_content.title:

# Example 1: print the title element of the page content
print("Example 1: get a specific HTML element - such as the page title")
print(page_content.title)

### A string corresponds to a bit of text within a tag.
### If a tag has only one child, and that child is a NavigableString, the child is made available as .string

# Example 2: print just the text of the title element using the 'text' operator
print("\nExample 2: get HTML element's text content")
print(page_content.title.string)

### A tag may have any number of attributes. The tag <div id="id"> has an attribute “id” whose value is “toolbar”. 
### You can access a tag’s attributes by treating the tag like a dictionary: div['id']

# Example 3: navigate along a tag sequence path and print content
print("\nExample 3: navigate along an element path to some content inside the page")
print(page_content.body.div['id'])

# Example 4: find ALL hyperlinks on the page (anywhere on the page, just print URL)
print("\nExample 4: find_all() URLs of hyperlinks on this webpage")
### The find_all() method looks through a tag’s descendants and retrieves all descendants that match your filters.
for link in page_content.find_all("a"):
    if (link.has_attr('href')):  
        print(link.get('href'));

Example 1: get a specific HTML element - such as the page title
<title>Batavia Convict Ship</title>

Example 2: get HTML element's text content
Batavia Convict Ship

Example 3: navigate along an element path to some content inside the page
toolbar

Example 4: find_all() URLs of hyperlinks on this webpage
https://convictrecords.com.au/convicts/create
https://convictrecords.com.au/bouncer/login
https://convictrecords.com.au/bouncer/register
https://convictrecords.com.au/
https://convictrecords.com.au/
https://convictrecords.com.au/
/convicts
/timeline
/ships
/crimes
/contributions
https://www.facebook.com/ConvictRecords
/resources
https://convictrecords.com.au/
https://convictrecords.com.au/ships
https://convictrecords.com.au/ships/b
https://convictrecords.com.au/ships/batavia/1817
https://convictrecords.com.au/
https://convictrecords.com.au/convicts/
https://convictrecords.com.au/about
https://convictrecords.com.au/facts
https://convictrecords.com.au/resources
https://www.facebook.com/C

## Exercise 1a:

Ok, now that we have learned a bit about how to navigate in a webpage, let's get to work:

What we would like to do in this exercise is to extract the ship voyage information from this page.

When you scroll around the HTML code, you will see a around the last third of the source code an element
> &lt;div id="ship"&gt; 

>    &lt;h1&gt;Batavia Voyages to Australia&lt;/h1&gt;
 
which describes the ship Batavia and its voyage to Australia in 1817 in a few more details.

Lets use Python to extract just this part.
To do so, we use the Beautiful Soup library to search for an HTML tag with id="ship", extract it from the page into a variable *shipdetails*, and print it out:

In [3]:
from bs4 import BeautifulSoup

page_content = BeautifulSoup(webpage_source, 'html5lib')
### Rather than passing in limit=1 every time you call find_all, you can use the find() method.
### The only difference is that find_all() returns a list containing the single result, and find() just returns the result.
shipdetails  = page_content.find(id="ship")
print(shipdetails)

<div id="ship">
    <h1>Batavia Voyages to Australia</h1>
    
    <div id="results">
        <ul>
                        <li>
                <img alt="Batavia" src="https://convictrecords.com.au/assets/css/images/ship-placeholder.jpg"/>

                <h3><a href="https://convictrecords.com.au/ships/batavia/1817">Sailed on October, 1817</a></h3>
                Arrived  5th April, 1818 at New South Wales<br/>
                Average sentence: 9 Years | Life
                sentences: 51 | Passengers: 220            </li>
                    </ul>
    </div>
</div>


This looks already like some useful information, but there is still a lot of HTML formatting code included here.
The **text** function allows us to just print all the printable text inside an HTML code fragment:

In [4]:
print(shipdetails.text)


    Batavia Voyages to Australia
    
    
        
                        
                

                Sailed on October, 1817
                Arrived  5th April, 1818 at New South Wales
                Average sentence: 9 Years | Life
                sentences: 51 | Passengers: 220            
                    
    



Ok, we are close. Now lets extract indiviudal parts of this text into some variables. To do so, we need to analyse and parse the actual HTML structure in which this text is contained.

Checking back with the HTML fragment, we note that the ship name is used as alternative text for the image of the ship. We can extract this with an **path expression** that looks for the first **img** tag in the ship details, and then extracts the value of its **alt** attribute.

In [5]:
shipname = shipdetails.img['alt']
print(shipname)

Batavia


There is a subheading (**h3**) in the text that tells us the sailing date.
This time, the text we are interested in is not an attribute of the element, but rather the text **string** inside the element. 
We do this as follows:

In [6]:
voyage = shipdetails.h3.string
print(voyage)

Sailed on October, 1817


We also can get the URL for the subpage with the details of that voyage from this h3 element:

In [7]:
voyagedetails = shipdetails.h3.a["href"]
print(voyagedetails)

https://convictrecords.com.au/ships/batavia/1817


Finally, let's get the information about when the ship arrived somewhere. This is a bit tricky because this part of the webpage has mixed content of a **h3** heading followed just by text without an enclosing HTML tag. If you draw the structure of our content fragment as a tree, you see that this text sits on the same level under the enclosing **div** element than the **h3** title in front of it. In the terminology of a tree, this is a **sibling** (the next sibling) to the h3-heading. 

So we need to tell the BeautifulSoup library to navigate along the HTML content exactly this way:
From the **div** element (pointed to by the *shipdetails* variable) on to the **h3** heading element, then to its next  sibling content element which is the text about the arrival date:

In [8]:
### You can use .next_sibling and .previous_sibling to navigate between page elements that are on the same level of the parse tree:
arrivaldate = shipdetails.h3.next_sibling.string
print(arrivaldate)


                Arrived  5th April, 1818 at New South Wales


The above code is actually a shortcut. The h3-heading and the text are actually contained in a list further inside the HTML tree. The complete correct path to this element is shown below - but luckily BeautifulSoup allows us to use a shortcut, interpreting each dot-stop (.) as a 'next level or any level below' direction:

In [10]:
print("Arrival date found along the complete HTML path:")
arrivaldate = shipdetails.div.ul.li.h3.next_sibling.string
print(arrivaldate)

Arrival date found along the complete HTML path:

                Arrived  5th April, 1818 at New South Wales


There seems to be some extra whitespace arount this arrival text; this can be removed using the **strip()** function on text.

Ok, putting this all together as just one code, we get the following web scraping program for Ship details.


In [11]:
import requests
from bs4 import BeautifulSoup

# retrieve the source code of the webpage
page_html = requests.get("https://convictrecords.com.au/ships/batavia/").text

# parse the HTML content of the webpage and extract the ship's details part
page_content = BeautifulSoup(page_html, 'html5lib')
shipdetails = page_content.find(id="ship")

# extract selected information from the shipdetails, based on page structure
shipname = shipdetails.img['alt']
voyage = shipdetails.h3.text
voyagedetails = shipdetails.h3.a["href"]
arrivaldate = shipdetails.h3.next_sibling.string

# print out retrieved values
print("Ship Name:  " + shipname)
print("Voyage:     " + voyage)
print("Voyage URL: " + voyagedetails)
print(arrivaldate.strip()) #remove extra whitespace from text

Ship Name:  Batavia
Voyage:     Sailed on October, 1817
Voyage URL: https://convictrecords.com.au/ships/batavia/1817
Arrived  5th April, 1818 at New South Wales


For more documentation about functions available in BautifulSoup, see here:
https://www.crummy.com/software/BeautifulSoup/bs4/doc/

## Exercise 1b:

What happens if a ship did more than just a single voyage?

In this case, its webpage would show a list of voyages - and our processing would need to be adjusted to be able to work with those lists.

Let's see on how this work on the example of the ship 'Adelaide' which did 3 voayges to Australia between 1849 and 1863:

In [12]:
import requests
from bs4 import BeautifulSoup

# retrieve the source code of the webpage
page_html = requests.get("https://convictrecords.com.au/ships/adelaide/").text

# parse the HTML content of the webpage and extract the ship's details part
page_content = BeautifulSoup(page_html, 'html5lib')
shipdetails = page_content.find(id="ship")
print(shipdetails)

<div id="ship">
    <h1>Adelaide Voyages to Australia</h1>
    <p>The ‘ADELAIDE’ was built 1832. Wood ship of 640 Tons.</p>
    <div id="results">
        <ul>
                        <li>
                <img alt="Adelaide" src="https://convictrecords.com.au/assets/css/images/ship-placeholder.jpg"/>

                <h3><a href="https://convictrecords.com.au/ships/adelaide/1849">Sailed on 8th August, 1849</a></h3>
                Arrived  29th November, 1849 at Van Diemen's Land and Port Phillip<br/>
                Average sentence: 8 Years | Life
                sentences: 11 | Passengers: 303            </li>
                        <li>
                <img alt="Adelaide" src="https://convictrecords.com.au/assets/css/images/ship-placeholder.jpg"/>

                <h3><a href="https://convictrecords.com.au/ships/adelaide/1855">Sailed on 16th April, 1855</a></h3>
                Arrived  18th July, 1855 at Western Australia<br/>
                Average sentence: 14 Years | Life
   

It seems that the format of the ConvictRecords web page is such that different voyages for the same ship are listed each with its own **h3** element followed by some text and links.
Our code above would so far only retrieve the first voyage though.
We need to adjust it such that it also works with multiple **h3** elements:

In [13]:
# extract selected information from the shipdetails, based on page structure
shipname = shipdetails.img['alt']
voyages = shipdetails.find_all("h3") # this finds ALL h3 elements describing voyages
voyage = []
voyurl = []
voyarrival = []
for v in voyages:
    voyage.append(v.text)
    voyurl.append(v.a["href"])
    voyarrival.append(v.next_sibling.string.strip())

# print out retrieved values
print("Ship Name: " + shipname)
print("The " + shipname + " made " + str(len(voyage)) + " voyages to Australia.")
#print(voyage)
for i in range(len(voyages)):
    print("Voyage "+str(i+1)+": " + voyage[i])
    print("     URL: " + voyurl[i])
    print("     "+voyarrival[i])

Ship Name: Adelaide
The Adelaide made 3 voyages to Australia.
Voyage 1: Sailed on 8th August, 1849
     URL: https://convictrecords.com.au/ships/adelaide/1849
     Arrived  29th November, 1849 at Van Diemen's Land and Port Phillip
Voyage 2: Sailed on 16th April, 1855
     URL: https://convictrecords.com.au/ships/adelaide/1855
     Arrived  18th July, 1855 at Western Australia
Voyage 3: Sailed on 13th May, 1863
     URL: https://convictrecords.com.au/ships/adelaide/1863
     Arrived                 (estimated) 13th January, 1864 at Gibraltar


## YOUR TASK: Extract the passenger list from the voyage of the Glory

In the lecture, we did briefly demonstrate how to extract a passenger list from a HTML table on a webpage on the ConvictRecords site. We used Unix tools there - your task here in this tutorial is to do the same with Python.

The **table** tag in HTML has the following general structure:

   tag names           | description
   :------------------:|:------------------------------------------------------
   **table**           | start tag of a HTML table
   **tr** ... **/tr**  | start and end tags of a table *row*
   **th** ... **/th**  | optional: start and end of a *header* field
   **td** ... **/td**  | td tags encapsulate a *single data field* within a table
   **/table**          | end tag of a table
   

Extract the *passenger list* from the **1818 journey** of the **ship Glory**, with the following constraints:
 - Extract the data into a Python **list** variable called **passengers** which is either a **list of strings** or better, a  **list of dictionary** data structure
 - As we already found out above, the url to this voyage is:<br>
   https://convictrecords.com.au/ships/glory/1818

In [16]:
# TODO: replace the content of this cell with your Python web scraping solution
# Example solution EXERCISE 1 import requests
from bs4 import BeautifulSoup

# retrieve the source code of the webpage
page_html = requests.get("https://convictrecords.com.au/ships/glory/1818").text

# parse the HTML content of the webpage and extract the voyage's de tails part
page_content = BeautifulSoup(page_html, 'html5lib')
data_table = page_content.find("table")
           #print(data_table)
# APPROACH 1: extract into a LIST of strings
# extract passenger list from data table, row by row, and add into a 'passengers' list of strings
passengers = []
data_rows = data_table.find_all("tr")
for row in data_rows:
    entry = ""
    for v in row.find_all("td"):
        if (entry == ""):
            entry = v.string
        else:
            entry += ", " + v.string
    if (entry):
        passengers.append(entry)   
for p in passengers:
    print(p)                                    

William Andrews, William Andrews, one of 170 convicts transported on the Glory, May 1818, Kent, Rochester City Quarter Session
Samuel Arnold, Samuel Arnold, one of 170 convicts transported on the Glory, May 1818, London Gaol Delivery
Daniel Arrowsmith, Daniel Arrowsmith, one of 170 convicts transported on the Glory, May 1818, Stafford Assizes
James Arundel, James Arundel, one of 170 convicts transported on the Glory, May 1818, Lancaster Quarter Session
Henry Ashley, Henry Ashley, one of 170 convicts transported on the Glory, May 1818, York, Kingston Upon Hull Quarter Session
John Ashton, John Ashton, one of 170 convicts transported on the Glory, May 1818, Lancaster Quarter Session
Thomas Astbury, Thomas Astbury, one of 170 convicts transported on the Glory, May 1818, Lancaster, Liverpool Borough Quarter Session
Thomas Astley, Thomas Astley, one of 170 convicts transported on the Glory, May 1818, Lancaster Quarter Session
Uriah Aust, Uriah Aust, one of 170 convicts transported on the Gl

In [19]:
# APPROACH 2: extract into a DICTIONARY LIST
# extract passenger list from data table, row by row, and add into a 'passengers' list of dictionary objects
# first retrieve the column headers
headers = []
for h in data_table.find_all("th"):
    headers.append(h.string.replace(" ", "_")) # important to remov e white spaces form field names for later processing
    # next, get the actual passenger list
    passengers = []
data_rows = data_table.find_all("tr") 
for row in data_rows:
    entry = {}
    column = 0
    for v in row.find_all("td"):
        entry[ headers[column] ] = v.string
        column += 1
    if (column == 3 ): passengers.append(entry) 
# we only take complete entries with data for all 3 columns
# print the passenger list
for p in passengers: print(p)     

{'Convict': 'William Andrews', 'Description': 'William Andrews, one of 170 convicts transported on the Glory, May 1818', 'Convicted_At': 'Kent, Rochester City Quarter Session'}
{'Convict': 'Samuel Arnold', 'Description': 'Samuel Arnold, one of 170 convicts transported on the Glory, May 1818', 'Convicted_At': 'London Gaol Delivery'}
{'Convict': 'Daniel Arrowsmith', 'Description': 'Daniel Arrowsmith, one of 170 convicts transported on the Glory, May 1818', 'Convicted_At': 'Stafford Assizes'}
{'Convict': 'James Arundel', 'Description': 'James Arundel, one of 170 convicts transported on the Glory, May 1818', 'Convicted_At': 'Lancaster Quarter Session'}
{'Convict': 'Henry Ashley', 'Description': 'Henry Ashley, one of 170 convicts transported on the Glory, May 1818', 'Convicted_At': 'York, Kingston Upon Hull Quarter Session'}
{'Convict': 'John Ashton', 'Description': 'John Ashton, one of 170 convicts transported on the Glory, May 1818', 'Convicted_At': 'Lancaster Quarter Session'}
{'Convict'

## Web Scraping with Google Spreadsheets

As a side note: If the data is given as a table or an item list, you can retrieve the structured data also in Google Spreadsheets using its **ImportHTML()** function:

In [ ]:
# login to Google Spreadsheet
# Type into the first cell:
# =ImportHTML("https://convictrecords.com.au/ships/glory/1818", "table", 1)

Note however that this **only works for table or list data** (so not for our initial example with the structured ship details), and that you would need to do so manually for each individual webpage. With a dataset of about 1200 different ship voyages, this is not really an option...

# EXERCISE 2: Data Cleansing

Once we retrieved data, we typically need to do some cleansing steps. This is especially important for data scrapped from webpages because many websites are not written with providing consistent data for a data analysis task in mind.

## Special Characters in Scrapped Data

The first issue we have to be aware of is that the scrapped data can include all kinds of special characters, some of which have special meaning in our target data models. 

For example, CSV files use commas (",") or tabs as a delimiter character between fields. If the text within one of such fields contains a comma (or tab) itself, the CSV file cannot be parsed correctly anymore. To avouid this, we need to **quote** all texts that potentially contain those special characters in CSV. Luckily, the csv write function in python allows us to do so as we will see later.

Another special character is the single quote ("'"). This one is used in SQL to encapsulate strings. But what if a string itself includes a single quote? we have then to *escape* it first.

Let's have a look on whether we find any single quote in our current passenger list from the ship **"Glory"**:

In [20]:
for p in passengers:
    if (p['Convict'].find("'") > 0): 
        print (p['Convict'])
    if (p['Description'].find("'") > 0): 
        print (p['Description'])
    if (p['Convicted_At'].find("'") > 0): 
        print (p['Convicted_At'])

James O'Donnell
James O'Donnell, one of 170 convicts transported on the Glory, May 1818


And indeed we seem to have one entry in this passenger list where a name with a aphostrophe is used. This is however the same character then the SQL single quote.

Now luckily, the python libraries for working with CSV files and with PostgreSQL are robust enough and will handle these special characters gracefully (at least with the utility functions which we provide below, and if you use dictionaries as data structures). **In our case, we do not need to do anything at this stage.**

In other cases however this might now require some pre-processing on your side such as escaping any single quotes or replacing them. It is also a good practice to use dictionary data structures rather than lists because they use attribute key names rather than relaying on positions.

## General Data Cleansing

For the next exercise, look at the 'Crimes' page on the ConvictRecords website:<br>
 https://convictrecords.com.au/crimes

This page is a good example for potential inconsistencies on a webpage, which we would like to avoid.

## YOUR TASK: Check and Discuss Data Inconsistencies in 'Crimes' list

- Write Python code to read the 'Crimes' page from the ConvictRecords website, and extract the crimes list (just textual would be fine):  https://convictrecords.com.au/crimes
- Check the content of this webpage and checkfor any data inconsistencies.
- Discuss with your class mates and tutor your findings - and how you would handle each case.

In [29]:
import requests
from bs4 import BeautifulSoup
webpage_source = requests.get("https://convictrecords.com.au/crimes/").text
page_content = BeautifulSoup(webpage_source, 'html5lib')
crimelist = page_content.find(id='results')
print(crimelist.text)


            
            Abuse
        
            
            Accessory to murder
        
            
            Arson
        
            
            Assault
        
            
            Assault and attempted robbery
        
            
            Assault and theft
        
            
            Assault with intent to rob with ano...
        
            
            Assault with intent to rob with another and previous conviction
        
            
            Attempted murder
        
            
            Bad notes
        
            
            Beastiality
        
            
            Being in company of felons
        
            
            Bigamy
        
            
            Breaking and entering and stealing
        
            
            Burglary
        
            
            Charged with an offence against the crown (smuggling)
        
            
            Child stealing
        
            
            Coining
        
  

### Some observations of data inconsitencies on this page:
#### There are some strange characters in some crime names (eg. a tilde '~' in 'Theft~grand larceny'); to improve consistency replace them with a simple whitespace.
#### A few crimes entries have a full stop at the end; to match the majority you should remove those. 
#### Some crime descriptions have a comma included; such texts need to be quoted when being stored in a CSV file.
#### Similar with the use of single quotes in some crime names and SQL.
#### There are several inconsistent spelling variants (eg. 'House breaking' versus 'Housebreaking' or "desertation from army" versus "destertation from army'") which would better be correct and replaced with just a single entry.
#### One incomplete entry? "Assault with intent to rob with ano..." Needs to be checked against source.
...

# EXERCISE 3: Data Storage
Lets finally look at how we can store the extracted data in either a CSV file or a database for further processing. 

For the **CSV output** use the **csv** library of Python.<br>
It is documented here: https://docs.python.org/2/library/csv.html

For the **PostgreSQL** database output, use the **psycopg2** library as we used already in Week 4.

## Exercise 3a: Writing Scraped Data to a CSV File

The first obvious export format for structured data, which we retrieved from a webpage, is a CSV file. Depending on which data structure you have chosen during data scraping in Exercise 1b (as a list or as a dictionary), this uses either the normal *writer* or the *DictWriter* function of the **csv** library (cf. https://docs.python.org/2/library/csv.html).

## YOUR TASK: Write Passenger List as a CSV File

Output your passenger list from the **passengers** variable as CSV file.
 - import the **csv** library
 - open a CSV file with a name of your choice
 - if your data is in a list of strings, use the standard **writer** from the **csv** library;<br>
   if your data is in dictionary objects, use the **DictWriter** from the **csv** library
 - make sure that you set the CSV delimiter to comma (',') and the quote char to "<br>
   **Note:** Setting a *quote char* is important because some of the texts in the data include commas; without quoting those texts correctly, the CSV file cannot be correctly parsed later because the 'comma' is also used as a delimiter between fields.
 - write a header row, followed by as many rows as you have passengers

In [30]:
import csv
# the following assumes the passenger list to be a list of dictionary objects
# the content of the header row is directly read from the dictionary data structure which makes this very flexible
with open('passengers_glory_1817.csv', 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=headers, delimiter=',', quotechar='"') 
    writer.writeheader() 
    for p in passengers:
        writer.writerow(p)

## Exercise 3b: Writing Scraped Data to a Database

In the next step, we are looking into how to store the scraped data directly into a PostgreSQL database. To do so, we copy the following two utility functions from Week 4 to connect to PostgreSQL and to execute an SQL command there from Python (actually, pgquery() is an improved version of pgexec() from Week 4).

Please edit in the following cell the variables **YOUR_UNIKEY** and **YOUR_PW** to match your own PostgreSQL login.

In [35]:
import psycopg2
import psycopg2.extras

def pgconnect():
    # please replace <your_unikey> and <your_SID> with your own details
    YOUR_UNIKEY = "jche6589"
    YOUR_PW     = "jche6589"
    try: 
        conn = psycopg2.connect(host='soit-db-pro-2.ucc.usyd.edu.au',
                                database='y18s1d2001_'+YOUR_UNIKEY,
                                user='y18s1d2001_'+YOUR_UNIKEY, 
                                password=YOUR_PW) 
        print('connected')
    except Exception as e:
        print("unable to connect to the database")
        print(e)
    return conn

def pgquery( conn, sqlcmd, args=None, silent=False, returntype='tuple'):
    """ utility function to execute some SQL query statement
        it can take optional arguments (as a dictionary) to fill in for placeholder in the SQL
        will return the complete query result as return value - or in case of error: None
        error and transaction handling built-in (by using the 'with' clauses)"""
    retval = None
    with conn:
        cursortype = None if returntype != 'dict' else psycopg2.extras.RealDictCursor
        with conn.cursor(cursor_factory=cursortype) as cur:
            try:
                if args is None:
                    cur.execute(sqlcmd)
                else:
                    cur.execute(sqlcmd, args)
                if (cur.description != None ):
                    retval = cur.fetchall() # we use fetchall() as we expect only _small_ query results
                if silent != False:
                    print("success: " + silent)
            except Exception as e:
                if e.pgcode != None and not(silent):
                    print("db read error: ")
                print(e)
    return retval

/anaconda3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


## YOUR TASK: Store Passenger List in Database
Your task is as follows:
* Create a table in your PostgreSQL database that is suitable for the passenger data retrieved from above's webpage.
* Insert the extracted information of the passengers of the Batavia in that table
* Query your database for how many passengers you have
* Advanced: query the database whether there are any passengers with the same name (any passenger name used more than once?);
* More Advanced: what are the top-10 common first names (you would need to either store first and last name in separate columns, or you sue the SUBSTR() function in SQL to get the frist name from the full names).

In [37]:
# 1st: login to database
conn = pgconnect()

# if you want to reset the table
pgquery (conn, "DROP TABLE IF EXISTS Passengers", None, "Reset Table Passengers")

# 2nd: ensure that the schema is in place
organisation_schema = """CREATE TABLE IF NOT EXISTS Passengers (
                         name VARCHAR(100),
                         description TEXT,
                         convicted_at TEXT
                   )"""
pgquery (conn, organisation_schema, None, "Create Table Passengers")

# 3nd: load data
# IMPORTANT: make sure the header line of CSV is without spaces!
insert_stmt = """INSERT INTO Passengers(name,description,convicted_at) VALUES (%(Convict)s, %(Description)s, %(Convicted_At)s)"""
for p in passengers:
    pgquery (conn, insert_stmt, p)
    
# 4th: query data
result = pgquery (conn, "SELECT COUNT(*) FROM Passengers", returntype='dict')
print("Number of passengers: " + str(result[0]['count']))

# What has happened to James O'Donnell?
param = {'name' : "James O'Donnell"}
result = pgquery (conn, "SELECT * FROM Passengers WHERE name = %(name)s", param);
print(result)

# 5th: query data
result = pgquery (conn, "SELECT name, COUNT(*) FROM Passengers GROUP BY name HAVING COUNT(*) > 1 ORDER BY count DESC")
print("Repeated names (and how often):")
print(result)

# 6th: query data
result = pgquery (conn, "SELECT SUBSTR(name,1,POSITION(' ' IN name)-1), COUNT(*) FROM Passengers GROUP BY SUBSTR(name,1,POSITION(' ' IN name)-1) ORDER BY 2 DESC, 1 LIMIT 10")
print("Most common first names (and their frequency):")
print(result)

unable to connect to the database
could not translate host name "soit-db-pro-2.ucc.usyd.edu.au" to address: nodename nor servname provided, or not known



UnboundLocalError: local variable 'conn' referenced before assignment

# EXERCISE 4: Using Web APIs

In the final exercise, we are looking at some examples on how to access web APIs which are specifically provided for program to retrieve data. The advantage is that the data is well defined - no distracting HTML tags in between.

But the services uses two different formats - either JSON or XML.

For **JSON**, we will use the standard language support in Python and its **request** library.<br>
For **XML**, we will use the **lxml** parser library.

### Example 1: U.S. Government Website Analytic API

First some JSON example from the U.S. government website analytics:

In [33]:
# The number of people who visited a U.S. government website using Internet Explorer 6.0 in the last 90 days
import requests
response = requests.get("https://analytics.usa.gov/data/live/ie.json")
print(response.json()['totals']['ie_version']['6.0'])

10676


Which version of Internet Explorer is used most at this moment when contacting the U.S. government website. For this, we need to look at the actual JSON response. For this it is helpful to have a 'pretty-print' of the corresponding JSON data which is returned by the analytics.use.gov website. The Python **json** library can do this for us:

In [34]:
# The raw response from the U.S. government website
import requests
response = requests.get("https://analytics.usa.gov/data/live/ie.json")

import json
print(json.dumps(response.json(), indent=4, sort_keys=True))

{
    "meta": {
        "description": "90 days of visits from Internet Explorer users broken down by version for all sites. (>100 sessions)",
        "name": "Internet Explorer"
    },
    "name": "ie",
    "query": {
        "dimensions": "ga:date,ga:browserVersion",
        "end-date": "yesterday",
        "filters": "ga:browser==Internet Explorer;ga:sessions>10",
        "max-results": 10000,
        "metrics": [
            "ga:sessions"
        ],
        "samplingLevel": "HIGHER_PRECISION",
        "sort": [
            "ga:date",
            "-ga:sessions"
        ],
        "start-date": "90daysAgo",
        "start-index": 1
    },
    "sampling": {
        "containsSampledData": false
    },
    "taken_at": "2018-05-14T19:09:53.703Z",
    "totals": {
        "ie_version": {
            "10.0": 3122608,
            "11.0": 307451354,
            "4.0": 12,
            "5.0": 1732,
            "6.0": 10676,
            "7.0": 21252005,
            "8.0": 1180692,
            "9

From above's data, it seems that IE version 11.0 is currently the most popular version of internet explorer used.
<br>Amazingly, there are still visits with IE 4.0 though...

### Example 2: ABS Population Clock API

The Australian Bureau of Statistics provides the following web API os a *Population Clock Web Service* which gives some statistics about the current Australian population. The meaning of the various fields are explained here: http://www.abs.gov.au/AUSSTATS/abs@.nsf/Latestproducts/1420.0.55.001Main%20Features2User%20Guide?opendocument&tabname=Summary&prodno=1420.0.55.001&issue=User%20Guide&num=&view=

In [38]:
import requests
import json
response = requests.get("http://www.abs.gov.au/api/demography/populationprojection")
print(json.dumps(response.json(), indent=4, sort_keys=True))

population = response.json()['popNow']
print("Current population in Australia: "+str(population))

{
    "attribution": "Australian Bureau of Statistics",
    "birthRate": "1 minute and 43 seconds",
    "copyRight": "Copyright Commonwealth of Australia",
    "deathRate": "3 minutes and 16 seconds",
    "growthRate": "1 minute and 24 seconds",
    "overseasMigrationRate": "2 minutes and 18 seconds",
    "popNow": "24935226",
    "projectionStartDate": "30 September 2017",
    "rateSecond": "84.23211910447282809019",
    "source": "Australian Demographic Statistics, September Quarter 2017 (cat. no. 3101.0)",
    "sourceURL": "http://www.abs.gov.au/ausstats/abs@.nsf/mf/3101.0",
    "timeStamp": "15 May 2018 13:04:32 AEST"
}
Current population in Australia: 24935226


### Example 3: Google Web API
Here's another example with parameters send to a web service:
The Google Wep-API allows to convert a location address to a GPS location (and some information more). The following example looks up the GPS location of the School of IT building at "1 Cleveland Street, Darlington, Australia":

https://developers.google.com/maps/documentation/geocoding/intro

In [39]:
# Lookup of a given address via Google Wep-API:
import requests
base_url = 'http://maps.googleapis.com/maps/api/geocode/json'
my_params= {'address': '1 Cleveland Street,Darlington,Australia','language':'en'}
response = requests.get(base_url, params = my_params)
results  = response.json()['results']
x_geo    = results[0]['geometry']['location']
print(x_geo['lng'], x_geo['lat'])

151.1939855 -33.8882368


https://maps.googleapis.com/maps/api/geocode/json?address=%221%20Cleveland%20Street,Darlington,Australia%22&language=%22en%22

#### You often want to send some sort of data in the URL’s query string. 
#### If you were constructing the URL by hand, this data would be given as key/value pairs in the URL after a question mark
e.g. httpbin.org/get?key=val
e.g. https://maps.googleapis.com/maps/api/geocode/json?address="1 Cleveland Street,Darlington,Australia"&language="en"
#### Requests allows you to provide these arguments as a dictionary of strings, using the params keyword argument. As an example, if you wanted to pass key1=value1 and key2=value2 to httpbin.org/get, you would use the following code:

## YOUR TASK: TODO
 - Use the Google Web-Api to check for the *GPS location* of the **Sydney Opera House**
 - Also retrieve the 'formatted_address'.<br>
   For this you might need to inspect first how the JSON response is structured: Do hence first a pretty-print of the corresponding JSON response data.
 - Tip: if you want to see a map for a given GPS location, try: https://www.latlong.net/

In [ ]:
# TODO: replace the content of this cell with your Python solution
raise NotImplementedErro

### Example 4: Web API returning XML

Some web APIs return data in **XML** format.
The easiest library to work with such kind of data in Python is called **lxml**.
Its documentation can be found here:<br>
http://lxml.de

In [ ]:
###### In the "Justice News" RSS feed maintained by the Justice Department, the number of items published on a Friday
from datetime import datetime
from lxml import etree
import requests
url = 'https://www.justice.gov/feeds/opa/justice-news.xml'
news= requests.get(url).content
doc = etree.fromstring(requests.get(url).content)
items = doc.xpath('//channel/item')
dates = [item.find('pubDate').text.strip() for item in items]
ts = [datetime.strptime(d[0:16], "%a, %d %b %Y") for d in dates]
# for weekday(), 4 correspond to Friday
print(len([t for t in ts if t.weekday() == 4]))

### Example 5: GitHub API
Some final more complex example, extracting some information from the meta-data of a GitHub repository.

In [ ]:
# From the lecture slides: list of programming languages used in PostgreSQL according to GitHub repositories
import requests, json
endpoint= "https://api.github.com/users/postgres/repos"
repos   = json.loads(requests.get(endpoint).text)

from dateutil.parser import parse
from collections     import Counter
dates = [parse(repo["created_at"]) for repo in repos]
month_counts = Counter(date.month for date in dates)
weekday_counts = Counter(date.weekday() for date in dates)

last_5_repositories = sorted(repos,
                             key=lambda r: r["created_at"],
                             reverse=True)[:5]
last_5_languages = [repo["language"]
                    for repo in last_5_repositories]
print(last_5_languages)


## Further Web API References

If you are further interested in exploring some web APIs, have a look at the following lists:

101 Data Journalist Challenges
https://github.com/stanfordjournalism/search-script-scrape

Tutorial on how to use New York Times API (needs registration with NYT)
https://stanford.edu/~vbauer/teaching/nyt.html

NSW Public Transport Events (needs registration)
https://opendata.transport.nsw.gov.au/dataset/public-transport-realtime-alerts-0

Twitter web API (needs registration):
https://developer.twitter.com/en/docs/basics/getting-started

# References

Books:
- Seppe van den Broucke and Bart Baesens: "Practical Web Scraping for Data Science", Springer 2018. (available electroinically via USYD library)

# End of Tutorial. Many Thanks.